# dependencies

In [ ]:
!pip install crewai
!pip install crewai-tools
!pip install groq
!pip install langchain_huggingface
!pip install langchain-groq
# batch 2
!pip install python-docx
!pip install docx2pdf
!pip install Spire.Doc
!pip install plum-dispatch==1.7.4
# part 3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 834.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.6 MB/s eta 0:00:00
   

## libries

In [ ]:
import crewai
import crewai_tools
from groq import Groq
from crewai import Agent
from langchain.llms import OpenAI

import os
from crewai import Agent, Task, Crew, Process

import string
from pypdf import PdfReader

from langchain_groq import ChatGroq
from spire.doc import *
from spire.doc.common import *

from docx import Document

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


## API keys

In [ ]:
Groq_api_key = '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'
from langchain_groq import ChatGroq

import os
api_key = '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'
os.environ["GROQ_API_KEY"] = api_key

## The LLM

In [ ]:
llm = ChatGroq(model="llama3-8b-8192",
               api_key=Groq_api_key)

## The Web scrappers

In [ ]:
import requests
import bs4


def openup():
    topic = input('topic: ')
    topic = topic.replace(' ','+')
    print(topic)
    return topic

def parse():
    topic = openup()
    link = 'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q='+topic+'&btnG='
    res = requests.get(link)
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text)

    a = 'gs_r gs_or gs_scl'
    b= 'gs_res_ccl_mid'
    all_ = soup.select('#'+b)
    some = all_[0].select('.gs_or')


    all_links = {}

    for i in range(10):
        topic = some[i].select('div > h3')[0].select('a')[0].getText()

        print(topic)
        try:

            mlink = some[i].select('.gs_or_ggsm')[0].select('a')[0].get('href')
    #         nam = some[i].select('div')[0].select('div')[0].select('div')[0].text
    #         print(nam)
            print(mlink)

        except:
            print('no topic for ', topic)
            mlink = ''

        all_links[topic] = mlink

    return all_links
# returns a lst that contains the names and links

def Sichub2(title):
    payload = { 'request': title}
    link = 'https://sci-hub.st/'
    # link = 'https://sci-hub.st/10.1109/icmla.2017.0-179'

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}


    a = requests.post(link, data=payload, headers = headers)

    soup = bs4.BeautifulSoup(a.text)
    content = soup.select('embed')[0].get('src').replace('#navpanes=0&view=FitH','').replace('//','/')
    contents= 'https:/' + content
    r = requests.get(contents, stream=True)

    title = title.replace(':','')
    with open(title +'.pdf', 'wb') as file:
        file.write(r.content)

    print('DOne downloading: ', title )
    return title+'.pdf'


def google(title, url):
    dashboardFile = requests.get(url, allow_redirects=True)

    for i in string.punctuation:
        title = title.replace(i,'')

    title = title +'.pdf'
    open(title, 'wb').write(dashboardFile.content)
    print('google:', title)
    reader = PdfReader(title)
    return title

# AVX ( still untested )

In [ ]:
down = []
def Avx():
  topic = openup()
  cat = classify(topic)
  link = 'https://arxiv.org/search/' + cat+ '?query=' + topic + '&searchtype=all&abstracts=show&order=-announced_date_first&size=50'
  res = requests.get(link)
  res.raise_for_status()
  soup = bs4.BeautifulSoup(res.text)
  a = soup.select('.arxiv-result')

  for i in range(10):
    b = a[i].select('p')
    topic = b[1].text
    url = a[i].select('span')[0].select('a')[0].get('href')
    topic = topic.replace('\n','')
    topic = topic.replace(' ','')
    topic = topic.replace("/","")


    dashboardFile = requests.get(url, allow_redirects=True)
    title = topic.replace(':','')
    name = topic+'.pdf'
    open(name, 'wb').write(dashboardFile.content)
    down.append(name)
    print('Done downloading:', title)

def classify(topic):
  from groq import Groq

  api_key = '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'

  client = Groq(api_key = api_key)
  completion = client.chat.completions.create(
      model="llama3-8b-8192",
      messages=[
          {
              "role": "user",
              "content": """ heyyy calssify this topic " """ +topic+ """ " appropriately and respond if it is related to the subjects: physics or computer Science or Mathematics or
              Quantitative Biology or Statistics or Electrical Engineering and Systems Science or Economics or Quantitative Finance
              limt your answer to just one subject
              give a very short output
              eg:
              output: physics"""
          }
      ],
      temperature=1,
      max_tokens=1024,
      top_p=1,
      stream=True,
      stop=None,
  )

  you = []
  for chunk in completion:
      you.append((chunk.choices[0].delta.content or ""))
  m = ''
  for j in you:
    m = m + j
  print(m)

  subjects = {'Physics':'physics',
 'Computer Science':'cs',
 'Mathematics':'math',
 'Quantitative Biology':'q-bio',
 'Statistics':'stat',
 'Electrical Engineering and Systems Science':'eess',
 'Economics':'econ',
 'Quantitative Finance':'q-fin',

 }
  for i in subjects.keys():
    if i in m:
      a = subjects[i]
      print(a)
  return (a)



## The ZIpper

In [ ]:


import zlib
import zipfile

def compress(file_names):
    print("File Paths:")
    print(file_names)

#     path = "C:/data/"
    # Select the compression mode ZIP_DEFLATED for compression
    # or zipfile.ZIP_STORED to just store the file
    compression = zipfile.ZIP_DEFLATED

    # create the zip file first parameter path/name, second mode
    zf = zipfile.ZipFile("Papers.zip", mode="w")
    try:
        for file_name in file_names:
            # Add file to the zip file
            # first parameter file to zip, second filename in zip
            zf.write(file_name, file_name, compress_type=compression)

    except FileNotFoundError:
        print("An error occurred")
    finally:
        # Don't forget to close the file!
        zf.close()

## The Tool

In [ ]:
from google.colab import userdata
from crewai_tools import PDFSearchTool



def ragtool(path):

  rag_tool = PDFSearchTool(pdf=path,
      config=dict(
          llm=dict(
              provider="groq", # or google, openai, anthropic, llama2, ...
              config=dict(
                  model="llama3-8b-8192",
                  # temperature=0.5,
                  # top_p=1,
                  # stream=true,
              ),
          ),
          embedder=dict(
              provider="huggingface", # or openai, ollama, ...
              config=dict(
                  model="BAAI/bge-small-en-v1.5",
                  #task_type="retrieval_document",
                  # title="Embeddings",
              ),
          ),
      )
  )
  return rag_tool

In [ ]:
# # Test Run
# rag_tool.run("what is car detection")

## The agentic workflow

In [ ]:

def agents():
  agent1 =Agent(
      role="Ai guru",
      goal="find the methodology and summarize it ",
      backstory=" you are an Ai enthuisiat who is very passonate abiut te topic of the paper",
      verbose=True,
      llm = llm,
      tools = [rag_tool]
  )

  task1 = Task(
      expected_output=" a summary of the methoflogi",
      description="you are to fins and summarisze the methodology of the paper",
      agent=agent1,
  )

  agent2 =Agent(
      role="A professional technical writer ",
      goal="find the methodology and summarize it ",
      backstory=" you are a very hardworking technical writer who doesn't overlook mistakes ",
      verbose=True,
      llm = llm,
      context = [agent1]
  )

  task2 = Task(
      expected_output=" a better written content gotten from agent 1",
      description="rewrite the output of agent 1 in a better way, remove all the numbers and everything in brackets  ",
      agent=agent1,
  )

  my_crew = Crew(agents=[agent1,agent2], tasks=[task1,task2])
  inputs ={"question":"give me an email from the file "}

  crew = my_crew.kickoff()
  return crew

# FInally

In [ ]:
# Joining it all Together
data = parse()

err = []


for i in data.keys():
    try:
        a = Sichub2(i)
        down.append(a)
    except:
        err.append(i)

for i in err:
    try:
        b = google(i,data[i])
        down.append(b)
    except:
        print(i)



topic: brain computer interface in ar
brain+computer+interface+in+ar
Brain–computer interface integrated with augmented reality for human–robot interaction
no topic for  Brain–computer interface integrated with augmented reality for human–robot interaction
A wearable brain–computer interface instrument for augmented reality-based inspection in industry 4.0
no topic for  A wearable brain–computer interface instrument for augmented reality-based inspection in industry 4.0
Towards BCI-based interfaces for augmented reality: feasibility, design and evaluation
https://inria.hal.science/hal-01947344/file/Manuscript.pdf
Augmented reality (AR) and brain-computer interface (BCI): Two enabling technologies for empowering the fruition of sensor data in the 4.0 era
no topic for  Augmented reality (AR) and brain-computer interface (BCI): Two enabling technologies for empowering the fruition of sensor data in the 4.0 era
An augmented-reality based brain-computer interface for robot control
no topic 

# Let it begin

In [ ]:
#down = [ 'Deep learning based car damage classification.pdf', 'Capturing car-following behaviors by deep learning.pdf', 'Car detection for autonomous vehicle LIDAR and vision fusion approach through deep learning framework.pdf', 'A survey on theories and applications for self-driving cars based on deep learning methods.pdf', 'A deep-learning–based antifraud system for car-insurance claims.pdf', 'Brain4cars Car that knows before you do via sensory-fusion deep learning architecture.pdf', 'Augmented reality meets deep learning for car instance segmentation in urban scenes.pdf', 'Deep learning-based car damage classification and detection.pdf']
you = []

for i in down:
  reader = PdfReader(i)
  if len(reader.pages) <30:
    try:
      rag_tool = ragtool(i)
      words = agents()
      you.append(words)
    except:
      pass
  else:
    print('too Long')



> Entering new CrewAgentExecutor chain...
Thought: I need to start by searching the PDF's content to understand the methodology of the paper.

Action: Search a PDF's content

Action Input: {'query': "methodology"}
 

Relevant Content:
Report on the commitee on methods of clinical examination in elec- troencephalography. Electroenceph. Clin. Neurophysiol. 10(370) (1958) [4] Kato, H., Billinghurst, M.: Marker tracking and hmd calibration for a video-based augmented reality conferencing system. In: Iwar, p. 85. IEEE Computer Society, Los Alamitos (1999) [5] Lenhardt, A., Kaper, M.: HJ Ritter. An adaptive p300-based online brain-computer interface. IEEE Transactions on Neural Systems and Rehabilitation Engineering: A Publication of The IEEE Engineering in Medicine and Biology Society 16(2), 121(2008) [6] Rebsamen, B., Teo, C.L., Zeng, Q., Ang Jr., M.H., Burdet, E., Guan, C., Zhang, H., Laugier, C.: Controlling a wheelchair indoors using thought. IEEE IntelligentSystems, 18–24 (2007) [7] 



> Entering new CrewAgentExecutor chain...
Thought: I need to start by searching the PDF's content for the methodology section.

Action: Search a PDF's content
Action Input: {"query": "methodology"} 

Relevant Content:
participants were provided with a feedback: one of the targets was highlighted in green for 2s. Given the fact that we did not perform any online classiﬁcation, the feedback was sham. The correct target was highlighted 80% of the time while one of the other two was highlighted the rest of the time in order to keep the participants motivated. Then, participants had a 2s break before the next trial started. The structure of the trials is illustrated in Figure 2. In total, participants had to focus on each target 10 times per run, in a random order. The session was divided into 4 runs, with one run per condition: (C1) targets displayed on screen, no HoloLens, (C2) targets displayed on screen, HoloLens switched- off, (C3) targets displayed on screen, HoloLens switched-on, (



> Entering new CrewAgentExecutor chain...
Thought: I need to start by searching the PDF content to gather information about the methodology used in the paper.

Action: Search a PDF's content

Action Input: {'query': 'methodology'}
 

Relevant Content:
Figure 14 Case study setup.

Figure 4 The setup of EEG data collection.

Figure 8 The schematic diagram of the control logic.

Thought: Ah, the search is a good start! I think I can gather more information about the methodology by searching for specific keywords.

Action: Search a PDF's content

Action Input: {'query': 'control logic'}
 

Relevant Content:
Figure 8 The schematic diagram of the control logic.

the 2D monitor to the 3D real world, we introduce a wearable augmented reality headset to provide human some augmented visual feedbacks in the corresponding to their inputs, making the above-proposed brain-computer interface a close-loop one. The rest of this section will present how to implement the four components of the proposed

## saving it

In [ ]:
!pip install python-docx
!pip install docx2pdf
!pip install Spire.Doc
!pip install plum-dispatch==1.7.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 6.8 MB/s eta 0:00:00


In [ ]:

from docx import Document
documents = Document()
file_name = 'AI_Summary'
documents.add_heading('AI Summary', 0)
for i in range(len(you)):
  documents.add_heading(down[i].replace('/content/',''), level=1)
  documents.add_paragraph(you[i].raw)

documents.save(file_name + '.docx')


from spire.doc import *
from spire.doc.common import *
document = Document()
document.LoadFromFile(file_name+ '.docx')
# Save the file to a PDF file
document.SaveToFile(file_name+ '.pdf', FileFormat.PDF)
document.Close()

testing